In [ ]:

import pandas as pd
import os


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import explained_variance_score, mean_absolute_percentage_error
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor

In [ ]:
#load all dfs form a folder and merge them on ID
""" path = "/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/deconfounded_but_age/"
for i, file in enumerate(os.listdir(path)):
    if i == 0:
        df = pd.read_csv(path + file)
    else:
        df = pd.merge(df, pd.read_csv(path + file), on='ID') """
df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/deconfounded_but_age/aseg.volume_aparc.volume_aparc.thickness.csv")
label_df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/age_label/all_ages.csv")

label_df = label_df[['ID', 'label_Age']]

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
for col in label_df.columns:
    print(col)

In [ ]:
#merge the dataframes on ID
merged_df = pd.merge(df, label_df, on='ID', how='inner')

In [ ]:
print(merged_df["label_Age"].value_counts())
merged_df.dropna(inplace=True)
merged_df = merged_df[:5000]


In [ ]:
y = merged_df["label_Age"]
X = merged_df.drop(["ID", "label_Age"], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
def analyze_age_predictions(y_true, y_pred):
    # Calculate basic error metrics
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # Calculate absolute differences for each prediction
    differences = np.abs(y_true - y_pred)
    
    # Calculate median absolute error
    median_ae = np.median(differences)
    
    # Calculate percentage of predictions within different year ranges
    within_1_year = np.mean(differences <= 1) * 100
    within_3_years = np.mean(differences <= 3) * 100
    within_5_years = np.mean(differences <= 5) * 100
    
    # Calculate age-specific errors
    age_errors = pd.DataFrame({
        'true_age': y_true,
        'predicted_age': y_pred,
        'absolute_error': differences
    })
    
    age_group_errors = age_errors.groupby(pd.cut(age_errors['true_age'], 
                                                bins=range(0, 101, 10)))['absolute_error'].mean()
    
    # Print results
    print(f"Average Absolute Error: {mae:.2f} years")
    print(f"Root Mean Square Error: {rmse:.2f} years")
    print(f"Median Absolute Error: {median_ae:.2f} years")
    print(f"\nPredictions within:")
    print(f"1 year: {within_1_year:.1f}%")
    print(f"3 years: {within_3_years:.1f}%")
    print(f"5 years: {within_5_years:.1f}%")
    
    # Create error distribution plot
    plt.figure(figsize=(12, 6))
    plt.hist(differences, bins=50, edgecolor='black')
    plt.title('Distribution of Age Prediction Errors')
    plt.xlabel('Absolute Error (Years)')
    plt.ylabel('Frequency')
    plt.axvline(mae, color='red', linestyle='dashed', label=f'Mean Error: {mae:.2f}')
    plt.axvline(median_ae, color='green', linestyle='dashed', label=f'Median Error: {median_ae:.2f}')
    plt.legend()
    plt.show()
    
    # Create scatter plot of true vs predicted ages
    plt.figure(figsize=(10, 10))
    plt.scatter(y_true, y_pred, alpha=0.5)
    plt.plot([0, 100], [0, 100], 'r--')  # Perfect prediction line
    plt.xlabel('True Age')
    plt.ylabel('Predicted Age')
    plt.title('True vs Predicted Age')
    plt.axis('equal')
    plt.grid(True)
    plt.show()
    
    # Print age group errors
    print("\nAverage Error by Age Group:")
    print(age_group_errors)
    
    return {
        'mae': mae,
        'rmse': rmse,
        'median_ae': median_ae,
        'within_1_year': within_1_year,
        'within_3_years': within_3_years,
        'within_5_years': within_5_years,
        'age_group_errors': age_group_errors
    }

In [ ]:
import numpy as np
from sklearn.metrics import balanced_accuracy_score, classification_report
import tabpfn as tp
# Split the dataset


# Initialize the classifier
clf = tp.TabPFNRegressor()
# Fit the model
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)

In [ ]:
# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

# Print comprehensive regression metrics
print("\nRegression Metrics:")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Explained Variance Score: {explained_var:.4f}")
print(f"Mean Absolute Percentage Error: {mape:.4f}")

In [ ]:
analyze_age_predictions(y_test, y_pred)

In [ ]:
xg_reg = XGBRegressor(random_state=42
)

# Fit the model
xg_reg.fit(
    X_train, 
    y_train,
)



In [ ]:

# Make predictions
y_pred = xg_reg.predict(X_test)

# Calculate regression metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
explained_var = explained_variance_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

# Print comprehensive regression metrics
print("\nRegression Metrics:")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R² Score: {r2:.4f}")
print(f"Explained Variance Score: {explained_var:.4f}")
print(f"Mean Absolute Percentage Error: {mape:.4f}")

In [ ]:
analyze_age_predictions(y_test, y_pred)